In [4]:
#Steven Cruickshank
#Module 6 Assignment

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import base64


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names
#Username and password

username = "aacuser"
password = "smokey"

shelter = AnimalShelter()


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
print(len(df.to_dict(orient='records')))
print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Steven Cruickshank - SNHU')
image_filename = 'logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Steven Cruickshank - SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Div(
        [
            
            html.B("First: "),
            "Select a rescue type:",
            html.Br(),
            dcc.RadioItems(
                id="radio_items_id",
                options=[
                    {"label": "Water Rescue", "value": "WR"},
                    {"label": "Mountain Rescue", "value": "MR"},
                    {"label": "Disaster Rescue", "value": "DR"},
                    {"label": "Reset", "value": "R"},
                ],
                labelStyle={"display": "inline-block"},
            ),
            html.Br(),
            html.Br(),
        ]),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        #Added features including page numbers and sorting features to make it more user-friendly for
        #the client
        row_selectable="single",
        sort_mode="multi",
        sort_action="native",
        selected_columns=[],
        selected_rows=[],
        page_current=0,
        page_size=10,

    ),
    html.Br(),
    html.Hr(),
    html.Div(
        id='map-id',
        className='col s12 m6',
            )
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    if index is None:
        row = 0
    else:
        row = index[0]
        
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
              center=[30.75, -97.48], zoom=10, children=[
                  dl.TileLayer(id="base-layer-id"),
                  #dl.Marker(postition=[dff.iloc[row, 14],dff.iloc[row,15]],
                   #        children=[
                    #           dl.Tooltip(dff.iloc[row,4]),
                     #          dl.Popup([
                      #             html.H1("Animal Name"),
                       #            html.P(dff.iloc[row,9])
                        #       ])
                         #  ])
              ])
    ]
  

app.run_server(debug=True)

10000
Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks'],
      dtype='object')
Dash app running on http://127.0.0.1:17084/
